<a href="https://colab.research.google.com/github/r-o-h-a-n-2004/tranfer-learning-using-fine-tuning-/blob/main/transfer%20learing%20with%20funing%20tuning%20mobilenet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow tensorflow-datasets --quiet

import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras import layers, models

In [ ]:
(ds_train, ds_val), ds_info = tfds.load(
    'cats_vs_dogs',
    split=['train[:80%]', 'train[80%:]'],
    as_supervised=True,
    with_info=True
)

IMG_SIZE = (160, 160)

def format_example(image, label):
    image = tf.image.resize(image, IMG_SIZE)
    image = image / 255.0
    return image, label

BATCH_SIZE = 32

train_batches = ds_train.map(format_example).batch(BATCH_SIZE).prefetch(1)
val_batches = ds_val.map(format_example).batch(BATCH_SIZE).prefetch(1)

In [3]:
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip('horizontal'),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
    layers.RandomContrast(0.1),
])

In [4]:
base_model = tf.keras.applications.MobileNetV2(
    input_shape=IMG_SIZE + (3,),
    include_top=False,
    weights='imagenet'
)
base_model.trainable = True  # Unfreeze all layers first

# Freeze all layers except the last 2 convolutional layers
for layer in base_model.layers[:-2]:
    layer.trainable = False

# Check which layers are trainable
print("Trainable layers:")
for i, layer in enumerate(base_model.layers):
    if layer.trainable:
        print(i, layer.name)

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
Trainable layers:
152 Conv_1_bn
153 out_relu


In [5]:
inputs = tf.keras.Input(shape=IMG_SIZE + (3,))
x = data_augmentation(inputs)
x = base_model(x, training=True)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(64, activation='relu')(x)
x = layers.Dense(32, activation='relu')(x)
outputs = layers.Dense(1, activation='sigmoid')(x)

model = tf.keras.Model(inputs, outputs)

In [6]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),  # Lower LR for fine-tuning
    loss='binary_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    train_batches,
    validation_data=val_batches,
    epochs=30  # Increase if you want
)

Epoch 1/30
582/582 ━━━━━━━━━━━━━━━━━━━━ 580s 972ms/step - accuracy: 0.5803 - loss: 0.7133 - val_accuracy: 0.9166 - val_loss: 0.3356
Epoch 2/30
582/582 ━━━━━━━━━━━━━━━━━━━━ 569s 978ms/step - accuracy: 0.8524 - loss: 0.3920 - val_accuracy: 0.9448 - val_loss: 0.1744
Epoch 3/30
582/582 ━━━━━━━━━━━━━━━━━━━━ 624s 982ms/step - accuracy: 0.8946 - loss: 0.2763 - val_accuracy: 0.9570 - val_loss: 0.1269
Epoch 4/30
582/582 ━━━━━━━━━━━━━━━━━━━━ 616s 1s/step - accuracy: 0.9091 - loss: 0.2341 - val_accuracy: 0.9613 - val_loss: 0.1083
Epoch 5/30
582/582 ━━━━━━━━━━━━━━━━━━━━ 567s 973ms/step - accuracy: 0.9185 - loss: 0.2095 - val_accuracy: 0.9650 - val_loss: 0.0955
Epoch 6/30
582/582 ━━━━━━━━━━━━━━━━━━━━ 612s 1s/step - accuracy: 0.9201 - loss: 0.1944 - val_accuracy: 0.9695 - val_loss: 0.0890
Epoch 7/30
582/582 ━━━━━━━━━━━━━━━━━━━━ 609s 1s/step - accuracy: 0.9291 - loss: 0.1838 - val_accuracy: 0.9703 - val_loss: 0.0851
Epoch 8/30
582/582 ━━━━━━━━━━━━━━━━━━━━ 568s 955ms/step - accuracy: 0.9320 - loss: 0.

KeyboardInterrupt: 